In [37]:
from deap import base, creator, gp, tools, algorithms
import operator, math, random
import numpy as np
import pygraphviz as pgv
import sympy

In [2]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)

In [4]:
reg = np.loadtxt("regression.txt", skiprows=2)
reg_x = reg[:,0].tolist()
reg_y = reg[:,1].tolist()
print(reg_x)
print(reg_y)

[-2.0, -1.75, -1.5, -1.25, -1.0, -0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75]
[37.0, 24.16016, 15.0625, 8.91016, 5.0, 2.72266, 1.5625, 1.09766, 1.0, 1.03516, 1.0625, 1.03516, 1.0, 1.09766, 1.5625, 2.72266, 5.0, 8.91016, 15.0625, 24.16016]


In [53]:
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error
    sqerrors = ((func(reg_x[i]) - reg_y[i])**2 for i in range(len(points)))
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=reg_x)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(0)

    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 20, stats=mstats,
                                   halloffame=hof, verbose=True) # P(mate), P(mutate), ngen
    # print log
    return pop, log, hof # returns population, log, hall of fame

progs = main()

   	      	                    fitness                    	                     size                     
   	      	-----------------------------------------------	----------------------------------------------
gen	nevals	avg    	gen	max    	min    	nevals	std    	avg  	gen	max	min	nevals	std   
0  	500   	150.345	0  	285.684	59.9551	500   	32.7096	4.916	0  	7  	3  	500   	1.9269
1  	265   	129.624	1  	754.98 	59.9551	265   	40.3795	4.944	1  	13 	1  	265   	2.19382
2  	267   	109.771	2  	285.684	59.9551	267   	26.7508	5.036	2  	15 	1  	267   	2.25005
3  	300   	103.906	3  	754.98 	56.7337	300   	46.5065	6.076	3  	13 	1  	300   	2.23119
4  	273   	98.6647	4  	3529.31	55.1298	273   	158.865	6.724	4  	15 	1  	273   	2.0813 
5  	265   	84.6099	5  	891.042	48.0824	265   	61.8096	7.036	5  	15 	1  	265   	2.14446
6  	270   	79.8591	6  	479.153	48.0824	270   	51.5587	7.444	6  	17 	1  	270   	2.49216
7  	291   	93.3416	7  	3948.58	24.4816	291   	199.538	7.628	7  	17 	1  	291   	2.35916
8  	254

In [54]:
nodes, edges, labels = gp.graph(progs[2][0]) # hof

print(gp.PrimitiveTree(progs[2][0]))

add(protectedDiv(x, x), mul(sub(x, mul(x, sub(add(x, x), mul(x, x)))), x))


In [11]:
g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.pdf")